In [1]:
import ipdb
import torch
import torch.nn as nn
import random 
import mmap
from torch.nn import functional as F
import pickle

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
print(torch.version.cuda)

cpu
11.8


In [2]:
# hyper parameters
max_iters = 5000
block_size = 64
batch_size = 64
learning_rate = 3e-4
eval_iters = 100
n_embd = 384
n_layer = 4
dropout = 0.1
n_head = 4

In [3]:
# load the data
chars = ""
with open('vocab.txt','r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(list(set(text)))
    
vocab_size = len(chars)

In [4]:
# tokenizer
# a tokeniser consists of an encoder and a decoder
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])
# data = torch.tensor(encode(text), dtype=torch.long)


In [5]:
# memory map for using small snippets of text from a single file of any size
def get_random_chunk(split):
    filename = "output_train.txt" if split == 'train' else "output_val.txt"
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            # Determine the file size and a random position to start reading
            file_size = len(mm)
            start_pos = random.randint(0, (file_size) - block_size*batch_size)

            # Seek to the random position and read the block of text
            mm.seek(start_pos)
            block = mm.read(block_size*batch_size-1)

            # Decode the block to a string, ignoring any invalid byte sequences
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')
            
            # Train and test splits
            data = torch.tensor(encode(decoded_block), dtype=torch.long)
            
    return data


In [6]:
def get_batch(split):
    data = get_random_chunk(split)
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [7]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [8]:
class Head(nn.Module):
    """ One self attention head """
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias = False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # inputs of size (batch, time-step, channels)
        # outputs of size (batch, time-step, head size)

        B, T, C = x.shape
        k = self.key(x) # (B, T, hs)
        q = self.query(x) # (B, T, hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # scaling down the dot product as very high correlations can be very dominant. This allows us to "hear" everything evenly
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B,T,T)
        wei = F.softmax(wei, dim=-1) # (B, T,T)
        wei = self.dropout(wei)

        # perform the weighted aggregation of the values
        v = self.value(x) # (B.T.hs)
        out = wei @ v
        return out

        

In [9]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-sttention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # concat along (B,T,C) -> C -> (B,T, [h1,h1,h2,h2,h3,h3])
        # concatenating makes it easier to process
        out = self.dropout(self.proj(out))
        return out

In [10]:
class FeedForward(nn.Module):
    """ a simple linear layer"""

    def __init__(self,n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4* n_embd, n_embd),
            nn.Dropout(dropout),
        )
    def forward(self, x):
        return self.net(x)

In [11]:
class Block (nn.Module):
    """ Transformeer blocks: communication folowed by computation """

    def __init__(self, n_embd, n_head):
        # n_embed: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head 
        # number of features each head will capture in the multiheadattention
        self.sa = MultiHeadAttention(n_head, head_size) # self attention
        self.ffwd = FeedForward(n_embd) 
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x+y)
        y = self.ffwd(x)
        x = self.ln2(x+y)
        return x

In [12]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embeddings_table = nn.Embedding(vocab_size, n_embd)
        # 8 different indices and each will be of size n_embd
        # using n_embd as we want to store information about the tokens in a large vector
        # this will have a bunch of attributes about a character
        # places semantic meaning on the characters
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])

        self.ln_f = nn.LayerNorm(n_embd) 
        self.lm_head = nn.Linear(n_embd, vocab_size) #vocab_size will give a prob distirubtion on OUR VOCAB

        self.apply(self._init_weights) #initialises the weights are standard deviations.

    # help the weights converge better
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            

    def forward(self, index, targets=None):

        B, T = index.shape
        # idx and targets are both (B,T) tensors of intergers
        tok_emb = self.token_embeddings_table(index)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) #(T,C)
        x = tok_emb + pos_emb# (B,T,C)
        x = self.blocks(x)# (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) 
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
        
    def generate(self, index, max_new_tokens):
        # index is (B X T) array of indices in the current context

        # generates based on the max new tokens
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            # focus on only the last time step
            logits = logits[:, -1, :] # this becomes B,C
            probs = F.softmax(logits, dim=-1)
            # sample from the distribution only take 1 so get the next one
            index_next = torch.multinomial(probs, num_samples=1) #(B,1)
            # append the sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) #(B, T+1)
        return index

model = GPTLanguageModel(vocab_size)
m = model.to(device)

# context = torch.zeros((1,1), dtype=torch.long, device=device)
# generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
# print(generated_chars)

In [13]:
# create the pytorch optimiser

optimiser = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# the training loop
for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.4f}, val loss: {losses['val']:.4f} ")

    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimiser.zero_grad(set_to_none=True) # zero grad ensures that the previous gradient does not affect the current one
    # the gradients are set to none instead of zero as this occupies less space
    loss.backward()
    optimiser.step()

print(loss.item())

with open('model-1.pkl','wb') as f:
    pickle.dump(model,f)

print('model saved')

step: 0, train loss: 9.2652, val loss: 9.2628 
step: 100, train loss: 2.3593, val loss: 2.4383 
step: 200, train loss: 2.3275, val loss: 2.3650 
step: 300, train loss: 2.2037, val loss: 2.2051 
step: 400, train loss: 2.0845, val loss: 2.0992 
step: 500, train loss: 2.0558, val loss: 1.9771 
step: 600, train loss: 2.0353, val loss: 1.9201 
step: 700, train loss: 1.8850, val loss: 1.8909 
step: 800, train loss: 1.8190, val loss: 1.7495 
step: 900, train loss: 1.8869, val loss: 1.8407 
step: 1000, train loss: 1.7766, val loss: 1.7771 
step: 1100, train loss: 1.7913, val loss: 1.7679 
step: 1200, train loss: 1.6987, val loss: 1.7480 
step: 1300, train loss: 1.7595, val loss: 1.7076 
step: 1400, train loss: 1.7713, val loss: 1.8583 
step: 1500, train loss: 1.6997, val loss: 1.7022 
step: 1600, train loss: 1.6426, val loss: 1.6416 
step: 1700, train loss: 1.6576, val loss: 1.6216 
step: 1800, train loss: 1.7046, val loss: 1.5994 
step: 1900, train loss: 1.5913, val loss: 1.6469 
step: 2000, 

In [14]:
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())
print(generated_chars)

NameError: name 'prompt' is not defined